### Setting up

BE WARNED! You will not be able to access smule web for at least half an hour when you run this!
Or just use VPN 😉

In [ ]:
# Dependencies
import json
import pandas as pd
import requests
from config import id_1, id_2

In [ ]:
# List of Song, Date, URL, inviter
# songs = []
# artist = []
# date = []
# inviter = []
# song_url = []
# recording_base_url = 'https://www.smule.com'

In [ ]:
# Create an empty list to store all the data output
mining_list = []

# Counter for while loop
n = 0

In [ ]:
# Building url
user_1 = f'https://www.smule.com/s/profile/performance/{id_1}/sing?offset='
user_2 = f'https://www.smule.com/s/profile/performance/{id_2}/sing?offset='
offset = 0

### Gather all the songs the two singers sang togehter

id_1 is the VIP

In [ ]:
# Infinite loop until the list is empty (offset ran out)

while True:
    response = requests.get(f'{user_1}{offset}')
    print(f'response code: {response.status_code}')
    
    # Get out of the loop if request is unsuccessful
    if response.status_code != 200:
        print('Unsuccessful Request T^T')
        break
    
    # Continue to data request
    response_json = response.json()
    json_stuff = response_json['list']
    n += 1
    print(f'Request #{n}: Success!')
    
    # If the list is empty in the json data
    if not json_stuff:
        break
    
    for x in range(0,len(json_stuff)):
        if json_stuff[x]['performed_by'].upper() == id_2.upper():
            mining_list.append(json_stuff[x])
#             inviter.append(json_stuff[x]['performed_by'])
#             songs.append(json_stuff[x]['title'])
#             artist.append(json_stuff[x]['artist'])
#             date.append(json_stuff[x]['created_at'])
#             recording_url = recording_base_url + json_stuff[x]['web_url']
#             song_url.append(recording_url)
            print(f'Processing record index #{offset + x}: {offset}, #{x}')

    offset = response_json['next_offset']
    
    # Get out of the while loop since there is no more data to request
    if offset == -1:
        print('No more records to be processed. Total Loops: {n}')
        break

In [ ]:
# Mapped all the wanted field from the data mining + rename field
def create_song_list(track):
    key = track['key']
    title = track['title']
    artist = track['artist']
    date = track['created_at']
    inviter = track['performed_by']
    recording_base_url = 'https://www.smule.com'
    recording_url = recording_base_url + track['web_url']
    
    return {
        'key': key,
        'Title': title,
        'Artist': artist,
        'Date_Created': date,
        'Invite_Spawner': inviter,
        'URL': recording_url
    }
    
# Use map to give a list iterator
song_list_iterator = map(create_song_list, mining_list)
# Turn iterator into a list
join_id1_list = list(song_list_iterator)

In [ ]:
# To dataframe and transform date string
join_id1 = pd.DataFrame(join_id1_list)
join_id1['Date_Created'] = join_id1['Date_Created'].str.split('T').str[0]

In [ ]:
# How many songs did id_2 join id_1 in total?
join_id1 = len(songs)
join_id1

In [ ]:
# Save first result to csv
join_id1.to_csv('../data/join_id1.csv', encoding='utf_8_sig', index=False)

id_2 is the VIP

In [ ]:
# Run another infinite loop again - You might have to wait 30mins b/c you'll get 418 code :P
# JUST USE VPN!

# Reset if uses different VPN to ping data
mining_list = [] 
offset = 0
n = 0

while True:
    response = requests.get(f'{user_2}{offset}')
    print(f'response code: {response.status_code}')
    
    # Get out of the loop if request is unsuccessful
    if response.status_code != 200:
        print('Unsuccessful Request T^T')
        break
    
    # Continue to data request
    response_json = response.json()
    json_stuff = response_json['list']
    n += 1
    print(f'Request #{n}: Success!')
    
    # If the list is empty in the json data
    if not json_stuff:
        break
    
    for x in range(0,len(json_stuff)):
        if json_stuff[x]['performed_by'].upper() == id_1.upper():
            mining_list.append(json_stuff[x])
#             inviter.append(json_stuff[x]['performed_by'])
#             songs.append(json_stuff[x]['title'])
#             artist.append(json_stuff[x]['artist'])
#             date.append(json_stuff[x]['created_at'])
#             recording_url = recording_base_url + json_stuff[x]['web_url']
#             song_url.append(recording_url)
            print(f'Processing record index #{offset + x}: {offset}, #{x}')

    offset = response_json['next_offset']
    
    # Get out of the while loop since there is no more data to request
    if offset == -1:
        print('No more records to be processed. Total Loops: {n}')
        break

TO BE CONTINUE!! RUNNING OUT OF TIME

In [ ]:
# How many songs did id_1 join id_2 in total?
join_id2 = len(songs) - join_id1
join_id2

In [ ]:
# Let's clean up the date column
date_created = []

for d in date:
    date_only = d.split('T')[0]
    date_created.append(date_only)

In [ ]:
# Show the list of songs in dataframe
all_songs_df = pd.DataFrame({'Date Created': date_created,
                             'Title':songs,
                             'Artist': artist,
                             'Invite Spawner': inviter,
                             'URL': song_url})

# Sort the df by date
all_songs_df = all_songs_df.sort_values('Date Created')

# Reset index
all_songs_df.reset_index(inplace=True)

# Delete old index
del all_songs_df['index']

### Final Product 🤩

In [ ]:
# Ta-da!
print(f'{id_1} joined {id_2} on {join_id2} songs!')
print(f'{id_2} joined {id_1} on {join_id1} songs!')
print(f'You guys have made {len(songs)} recordings together as of this date.')

all_songs_df

In [ ]:
# Save as a csv with encoding that allows foreign character
all_songs_df.to_csv('all_songs.csv', encoding='utf_8_sig')